# Lição 7
- Autor: Bruno Martins Valério Bomfim
- Matrícula: 211039297
- Disciplina: Aprendizado de Máquina 35M34
- Semestre: 2023.1

## 1. Introdução
O presente artigo almeja aplicar uma abordagem prática para o entendimento de conceitos absorvidos sobre filtros colaborativos durante a lição 7 do curso "**FastAI - Machine Learning for Coders**" com Jeremy Howard através da construção de um modelo simples de recomendação de livros para usuários de uma determinada plataforma.

## 2. Pré-requisitos
Para o artigo em questão, serão utilizadas bibliotecas referentes ao fastai e outras essenciais para tratar de operações com tensores e importação de arquivos.

In [4]:
import os
from fastai.collab import *
from fastai.tabular.all import *
set_seed(42)

## 3. Dados
Extraído do próprio kaggle, o [Book Recommendation Dataset](https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Ratings.csv) contém informações sobre usuários, livros e avaliações da Amazon, trazendo a seguinte descrição sobre o conteúdo dos arquivos CSV:

*The Book-Crossing dataset comprises 3 files.*

- *Users: Contains the users. Note that user IDs (User-ID) have been anonymized and map to integers. Demographic data is provided (Location, Age) if available. Otherwise, these fields contain NULL-values.*

- *Books: Books are identified by their respective ISBN. Invalid ISBNs have already been removed from the dataset. Moreover, some content-based information is given (Book-Title, Book-Author, Year-Of-Publication, Publisher), obtained from Amazon Web Services. Note that in case of several authors, only the first is provided. URLs linking to cover images are also given, appearing in three different flavours (Image-URL-S, Image-URL-M, Image-URL-L), i.e., small, medium, large. These URLs point to the Amazon web site.*

- *Ratings: Contains the book rating information. Ratings (Book-Rating) are either explicit, expressed on a scale from 1-10 (higher values denoting higher appreciation), or implicit, expressed by 0.*




Vamos adicioná-lo ao notebook:

In [5]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle: path = Path('../input/book-recommendation-dataset')
else:
    import zipfile,kaggle
    path = Path('book-recommendation-dataset')
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

books = pd.read_csv(path/'Books.csv')
ratings = pd.read_csv(path/'Ratings.csv')
users = pd.read_csv(path/'Users.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### 3.1. Aparência geral dos datasets

#### 3.1.1. Livros

In [6]:
books.head(6)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0195153448.01.LZZZZZZZ.jpg
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0002005018.01.LZZZZZZZ.jpg
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0060973129.01.LZZZZZZZ.jpg
3,0374157065,Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0374157065.01.LZZZZZZZ.jpg
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0393045218.01.LZZZZZZZ.jpg
5,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/0399135782.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/0399135782.01.LZZZZZZZ.jpg


In [7]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [8]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0195153448,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/185326119X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/185326119X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/225307649X.01.LZZZZZZZ.jpg
freq,1,27,632,13903,7535,2,2,2


#### 3.1.2. Avaliações (Ratings)

In [9]:
ratings.head(6)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
5,276733,2080674722,0


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [11]:
ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


#### 3.1.3. Usuários

In [12]:
users.head(6)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
5,6,"santa monica, california, usa",61.0


In [13]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [14]:
users.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [15]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


## 3. Construindo os filtros colaborativos
Antes de construir o filtro em si, precisamos mesclar as tabelas e criar um dataloader.

In [26]:
ratings = ratings.merge(books)
ratings.head(3)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.01.THUMBZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.MZZZZZZZ.jpg,http://images.amazon.com/images/P/034545104X.01.LZZZZZZZ.jpg


In [17]:
data_loader = CollabDataLoaders.from_df(ratings, item_name='Book-Title')
data_loader.show_batch()

,User-ID,Book-Title,Book-Rating
0,248280,The Liar's Club: A Memoir,7
1,103304,Murder at the National Gallery (Capital Crime Mysteries),0
2,30735,Emily of Deep Valley (Deep Valley Book),10
3,235842,"Freshman Flames (Freshman Dorm, No 13)",0
4,24937,The Art of Seduction,8
5,96055,What Smart Women Know,0
6,48046,Seize the Night,9
7,53123,Spider's Web,5
8,110373,The Best of Sisters in Crime,7
9,55492,Winter Rose,0


Com o dataloader preparado, podemos criar o nosso modelo utilizando a classe collab_learner():

In [20]:
learner = collab_learner(data_loader, n_factors=10, y_range=(0, 10.5))
learner.fit_one_cycle(5, 5e-2, wd=0.1)

epoch,train_loss,valid_loss,time
0,14.437444,14.347594,02:34
1,14.496790,14.346169,02:31
2,14.008732,14.087223,02:37
3,13.898979,13.720404,02:38
4,13.388379,13.415916,02:37


Os nomes das camadas podem ser visualizados abaixo:

In [21]:
learner.model

EmbeddingDotBias(
  (u_weight): Embedding(81473, 10)
  (i_weight): Embedding(241072, 10)
  (u_bias): Embedding(81473, 1)
  (i_bias): Embedding(241072, 1)
)

Pesquisando pelo viés, podemos ver os livros mais admirados e notados pelo público em geral.

In [25]:
book_bias = learner.model.i_bias.weight.squeeze()
idxs = book_bias.argsort(descending=True)[:5]
[data_loader.classes['Book-Title'][i] for i in idxs]

['Harry Potter and the Prisoner of Azkaban (Book 3)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Free',
 "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
 "Ender's Game (Ender Wiggins Saga (Paperback))"]

## 4. Inferência
Iremos comparar o desempenho de diferentes filtros através de uma implementação manual, e também variando a quantidade de fatores latentes e a taxa de aprendizadem (*learning rate*).

### 4.1. Criando um filtro colaborativo manualmente
Será utilizada a técnica do produto escalar (*Dot product*), considerando o *Weight Decay* e os viés (*Bias*).

Agora, vamos criar os vetores que conterão os fatores latentes. Eles serão inicializados com números pseudo-aleatórios.

In [30]:
number_of_users = len(data_loader.classes["User-ID"])
number_of_books = len(data_loader.classes["Book-Title"])
factors = 5

user_factors = torch.randn(number_of_users, factors)
book_factors = torch.randn(number_of_books, factors)
number_of_users, number_of_books, factors, user_factors.shape, book_factors.shape

(81473, 241072, 5, torch.Size([81473, 5]), torch.Size([241072, 5]))

Para criar a classe que fará o produto escalar, serão utilizados os seguintes recursos:

- **Função sigmóide**: converter os valores para um intervalo determinado. Neste caso, entre 0 e 10.
- **Bias**: vetor de pesos que irá balancear o resultado com os aspectos gerais de cada usuário e cada livro. E.g., um determinado livro pode ter baixa aprovação geral (i.e. pouco admirado pelo público), e um usuário pode conceder notas baixas para todos os livros.
- **Weight Decay**: adicionar a soma de todos os pesos ao quadrado.



In [56]:
class DotProduct(Module):
    def __init__(self, n_users, n_books, factors, range=(0, 10.5)):
        self.user_factors = Embedding(n_users, factors)
        self.user_bias = Embedding(n_users, 1)
        self.book_factors = Embedding(n_books, factors)
        self.book_bias = Embedding(n_books, 1)
        self.range = range
        
    def forward(self, x):
        users = self.user_factors(x[:,0])
        books = self.book_factors(x[:,1])
        total = (users * books).sum(dim=1, keepdim=True)
        total += self.user_bias(x[:,0]) + self.book_bias(x[:,1])
        return sigmoid_range(total, *self.range)

In [59]:
for fac in [5, 25, 50]:
    model = DotProduct(number_of_users, number_of_books, fac)
    manual_learner = Learner(data_loader, model, loss_func=MSELossFlat())
    manual_learner.fit_one_cycle(3, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,13.812664,13.664605,02:28
1,13.482218,13.554325,02:26
2,13.541309,13.494868,02:26


epoch,train_loss,valid_loss,time
0,13.390235,13.567513,02:51
1,13.333696,13.364586,02:51
2,13.336976,13.259862,02:49


epoch,train_loss,valid_loss,time
0,13.747203,13.584617,03:36
1,13.353620,13.306532,03:35
2,12.875508,13.183279,03:34


### 4.2. Variando parâmetros da classe collab_learner()

In [ ]:
for fac in [5, 25, 50]:
    learner = collab_learner(data_loader, n_factors=fac, y_range=(0, 10.5))
    learner.fit_one_cycle(6, 5e-3, wd=0.2)

epoch,train_loss,valid_loss,time
0,15.177421,15.198632,02:20
1,15.319475,15.218284,02:23
2,15.209715,15.208314,02:20
3,15.170794,15.139248,02:21
4,14.814439,15.068766,02:21
5,15.240605,15.043963,02:21


epoch,train_loss,valid_loss,time
0,14.924408,14.949375,02:46
1,15.080113,15.040168,02:46
2,15.064045,14.954975,02:44
3,14.771515,14.819196,02:44
4,14.775126,14.664204,02:45
5,14.486411,14.597652,02:46


epoch,train_loss,valid_loss,time


## Bibliografia
[Jeremy Howard and Sylvain Gugge - FastBook](https://github.com/fastai/fastbook)

[Collaborative Filtering Deep Dive](https://www.kaggle.com/code/jhoward/collaborative-filtering-deep-dive/notebook)